In [4]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from IPython.display import Javascript
from fuzzywuzzy import process, fuzz

In [5]:
import platform
import psutil
import os

In [6]:
server  = "168.63.44.210"
port    = "7687"
user    = "neo4j"
passw   = "digityser"
db_name = "neo4j"        # default name (v4.x)

In [7]:
# load graph connection instance
graph = Graph(host=server, auth=(user, passw), name=db_name, encrypted=False)

#### detected nodes that have no value

In [29]:
query = """
MATCH (s)
WHERE s.description STARTS WITH 'No '
RETURN s.description AS data
"""
graph.run(query).to_data_frame()

,data
0,No Feeling
1,No Satisfaction
2,No Improvement


In [30]:
query = """
MATCH (s)
WHERE s.description STARTS WITH 'No '
DETACH DELETE s
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 3
properties_set: 0
relationships_created: 0
relationships_deleted: 211

## what percentages has a positive, negative and neutral sentiment ?

In [31]:
%%time
query = """
MATCH (s:Sentiment {value:'positive'})-[:SENTIMENT_IN]->()<-[rel]-(e:Employee)
WITH count(s) AS totalPositive
MATCH (s:Sentiment {value:'negative'})-[:SENTIMENT_IN]->()<-[rel]-(e:Employee)
WITH totalPositive, count(s) AS totalNegative
MATCH (s:Sentiment {value:'neutral'})-[:SENTIMENT_IN]->()<-[rel]-(e:Employee)
WITH totalPositive, totalNegative, count(s) AS totalNeutral
MATCH (s:Sentiment)-[:SENTIMENT_IN]->()<-[rel]-(e:Employee)
WITH totalPositive, totalNegative, totalNeutral, count(s) AS total
RETURN toFloat((totalPositive)/toFloat(total))*100 AS pos,
toFloat((totalNegative)/toFloat(total))*100 AS neg,
toFloat((totalNeutral)/toFloat(total))*100 AS neu
"""
graph.run(query).to_data_frame()

CPU times: user 2.05 ms, sys: 573 µs, total: 2.62 ms
Wall time: 76.3 ms


,pos,neg,neu
0,49.438202,13.483146,37.078652


#### List all the entities related to positive, negative and neutral sentiment

In [32]:
# neutral entities:
query = """
MATCH (s:Sentiment {value:'positive'})-[:SENTIMENT_IN]->(feeling:Feeling)
WITH feeling
MATCH (s:Sentiment {value:'positive'})-[:SENTIMENT_IN]->(satisfaction:Satisfaction)
WITH feeling, satisfaction
MATCH (s:Sentiment {value:'positive'})-[:SENTIMENT_IN]->(suggestion:Suggestion)
WITH feeling, satisfaction, suggestion
MATCH (satisfaction)<-[r:SKILL_IN]-(d)
RETURN DISTINCT d.value AS positive_entities
"""
graph.run(query).to_data_frame()

,positive_entities
0,collecting
1,energy
2,openness
3,impact
4,certification
5,scrum
6,CSR
7,collaboration
8,communication


In [22]:
# negative entities:
query = """
MATCH (s:Sentiment {value:'negative'})-[:SENTIMENT_IN]->(feeling:Feeling)
WITH feeling
MATCH (s:Sentiment {value:'negative'})-[:SENTIMENT_IN]->(satisfaction:Satisfaction)
WITH feeling, satisfaction
MATCH (s:Sentiment {value:'negative'})-[:SENTIMENT_IN]->(suggestion:Suggestion)
WITH feeling, satisfaction, suggestion
MATCH (satisfaction)<-[r:SKILL_IN]-(d)
RETURN DISTINCT d.value AS negative_entities
"""
graph.run(query).to_data_frame()

,negative_entities
0,onboarding


In [21]:
# neutral entities:
query = """
MATCH (s:Sentiment {value:'neutral'})-[:SENTIMENT_IN]->(feeling:Feeling)
WITH feeling
MATCH (s:Sentiment {value:'neutral'})-[:SENTIMENT_IN]->(satisfaction:Satisfaction)
WITH feeling, satisfaction
MATCH (s:Sentiment {value:'neutral'})-[:SENTIMENT_IN]->(suggestion:Suggestion)
WITH feeling, satisfaction, suggestion
MATCH (satisfaction)<-[r:SKILL_IN]-(d)
RETURN DISTINCT d.value AS neutral_entities
"""
graph.run(query).to_data_frame()

,neutral_entities
0,training
1,impact
2,valuate
3,management decision
4,financial
5,management
6,micro management
7,continuous learning
